In [5]:
from typing import List
from collections import Counter
from typing import NamedTuple

# К ближайших соседей
# https://github.com/joelgrus/data-science-from-scratch/blob/master/scratch/k_nearest_neighbors.py

Vector = List[float]
###

def dot(v: Vector, w: Vector) -> float:
    """Computes v_1 * w_1 + ... + v_n * w_n"""
    assert len(v) == len(w), "vectors must be same length"

    return sum(v_i * w_i for v_i, w_i in zip(v, w))

assert dot([1, 2, 3], [4, 5, 6]) == 32  # 1 * 4 + 2 * 5 + 3 * 6

def sum_of_squares(v: Vector) -> float:
    """Returns v_1 * v_1 + ... + v_n * v_n"""
    return dot(v, v)

assert sum_of_squares([1, 2, 3]) == 14  # 1 * 1 + 2 * 2 + 3 * 3

import math

def magnitude(v: Vector) -> float:
    """Returns the magnitude (or length) of v"""
    return math.sqrt(sum_of_squares(v))   # math.sqrt is square root function

assert magnitude([3, 4]) == 5

def squared_distance(v: Vector, w: Vector) -> float:
    """Computes (v_1 - w_1) ** 2 + ... + (v_n - w_n) ** 2"""
    return sum_of_squares(subtract(v, w))

def distance(v: Vector, w: Vector) -> float:
    """Computes the distance between v and w"""
    return math.sqrt(squared_distance(v, w))


def distance(v: Vector, w: Vector) -> float:  # type: ignore
    return magnitude(subtract(v, w))
###

def raw_majority_vote(labels):
    """
    Грубый подсчет большинства голосов
    """
    votes = Counter(labels)
    winner, _ = votes.most_common(1)[0]
    
    return winner

assert raw_majority_vote(['a', 'b', 'c', 'b']) == 'b'


def majority_vote(labels):
    """
    Подразумевает, что метки упорядочены от ближайшей до самой удаленной
    """
    vote_counts = Counter(labels)
    winner, winner_count = vote_counts.most_common(1)[0]
    num_winners = len([count for count in vote_counts.values() if count == winner_count])
    # если победитель один - возвращаем его     
    if num_winners == 1:
        return winner
    
    return majority_vote(labels[:-1])


# Tie, so look at first 4, then 'b'
assert majority_vote(['a', 'b', 'c', 'b', 'a']) == 'b'


class LabeledPoint(NamedTuple):
    point: Vector
    label: str

        
def knn_classify(k: int, labeled_points: List[LabeledPoint], new_point: Vector) -> str:
    """
    каждая маркированная точка должна быть представлена парой (точка, метка)
    """
#     TODO: переписать по человечески!!!
    # упорядочить маркированные точки, от ближайшей до самой удаленной     
#     by_distance = sorted(labeled_points, key=lambda(point, _): distance(point, new_point))
    by_distance = sorted(labeled_points, key=lambda lp: distance(lp.point, new_point))
    
    # найти метки для k-ближайших
    k_nearest_labels = [label for _, label in by_distance[:k]]
    
    # и дать им проголосовать
    return majority_vote(k_nearest_labels)